# PLEASE ONLY RUN THE NEXT FOUR CELLS ONCE!

## Google Colab Runtime Configuration
**Before running any cells, please change the runtime version:**
1. Go to **Edit** → **Notebook Settings** → **Runtime Version** 
2. Select **2025.07** 
3. Click **Save**

## Important Notes:
- The setup cells below will install dependencies and may cause kernel restarts
- This is expected behavior - simply continue after each restart
- Only run the setup cells once per session

In [ ]:
# Global variable to check if we're in Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    ! pip install numpy==1.26.4
    ### The runtime will crash/restart after this

In [ ]:
# Global variable to check if we're in Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:

  !pip install -q condacolab #-U "git+https://github.com/mayankchetan/condacolab.git@py312update"
  import condacolab
  condacolab.install() #### expect a kernel restart

In [ ]:
# Global variable to check if we're in Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !git clone  https://github.com/wisdem/weis
    !git clone https://github.com/mayankchetan/weisWorkshop

    %cd weis
    !conda config --env --add pinned_packages "numpy<2.0"
    !mamba env update -n base -f environment.yml "python<3.13"
    %pip install .
    %cd ..
    %pip install "rosco==2.9.5"
    %pip install -U "git+https://github.com/NREL/ROSCO@v2.9.7"
    %pip install "orbit-nrel<1.2.2"
    %pip install gdown

else:
    try:
        import weis

    except ImportError:
        raise ImportError("Please install the weis package from https://github.com/wisdem/weis in the environment you are using to run this notebook.")

    Warning("Make sure you have cloned 'https://github.com/mayankchetan/weisWorkshop-Private' repository and are running the notebook from the root directory of the repository.")


In [ ]:
# Downloading the precomputed data from Google Drive & extracting
# Drive link for non Google Colab users: https://drive.google.com/file/d/145BH393nzpT2msCVsHSBxR3Qy8KdpddI/view?usp=sharing
if IN_COLAB:
    !gdown 145BH393nzpT2msCVsHSBxR3Qy8KdpddI -O /content/weisWorkshop/archive_weisWorkshop_OCT25.tar.gz
    !tar -xvzf /content/weisWorkshop/archive_weisWorkshop_OCT25.tar.gz -C /content/weisWorkshop/
else:
    print("Please download the precomputed data from the link given in the notebook and extract it in the root directory of the repository.")
    print("you can use `tar -xvzf <downloaded_file_name>` to extract the files.")

# Chapter 2: Controller Tuning and Platform Optimization

## Introduction to Multi-Fidelity (Co-)Design

In Chapter 1, we optimized our rotor using WISDEM's steady-state models. Now we advance to WEIS's full multi-fidelity capabilities, integrating dynamic analysis tools for comprehensive system optimization.

WEIS enables a systematic progression through modeling fidelities:
- **WISDEM**: Steady-state system engineering models for initial design
- **RAFT**: Frequency-domain analysis for rapid floating platform evaluation
- **OpenFAST**: Time-domain aeroelastic simulation for medium-fidelity load analysis

## Chapter 1 Recap

We will build on our previous work, where we:
- Established baseline performance of the IEA 22MW floating wind turbine
- Scaled the design to create a 20MW turbine (with a smaller, 270m rotor diameter)
- Optimized rotor aerodynamics and structure using WISDEM
- Analyzed optimization convergence and performance improvements

## Chapter 2 Objectives

Now we'll complete the integrated system design through multi-stage optimization:

### Stage 2: Controller Re-tuning with ROSCO
- **Challenge**: Our optimized rotor requires a retuned controller to operate efficiently and manage loads properly
- **Approach**: Use ROSCO within WEIS to optimize controller parameters for the new rotor design
- **Tools**: OpenFAST + ROSCO for analysis

### Stage 3a: Floating Platform Optimization with RAFT
- **Challenge**: Design an optimal semisubmersible platform for the 20MW system
- **Approach**: Use RAFT's frequency-domain analysis for rapid platform sizing and optimization
- **Tools**: RAFT for efficient floating platform design iteration

### Stage 3b: Medium-Fidelity System Integration
- **Challenge**: Validate and refine the design using high-fidelity dynamic analysis
- **Approach**: OpenFAST-based optimization incorporating controller and platform interactions
- **Tools**: OpenFAST with Design Load Cases (DLCs) for comprehensive load assessment

<!-- ### Stage 4: Performance Comparison
- Compare the baseline IEA 22MW with our fully optimized 20MW system
- Evaluate improvements in cost of energy, structural efficiency, and load management -->

This multi-stage approach demonstrates WEIS's systematic design methodology, progressing from rapid conceptual design to detailed engineering analysis.

In [ ]:
if IN_COLAB:
    # Now we try loading the weis_workshop module again
    sys.path.append('/content/weisWorkshop/jupyterNotebooks')

# Importing nesscary libraries.
from weis import weis_main
import os

from weis_workshop import *

# set env variables
os.environ["OMP_NUM_THREADS"] = "1"  # export OMP_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = "1"  # export MKL_NUM_THREADS=1

# workshop repository folder
if IN_COLAB:
    WORKSHOP_REPO = '/content/weisWorkshop'
else:
    WORKSHOP_REPO = os.path.join(os.getcwd(), '..')

## Multi-Stage Optimization Workflow

We'll now implement the multi-stage optimization approach outlined in our objectives. Each stage builds upon the previous work, demonstrating the systematic progression through WEIS's modeling capabilities.

**Computational Notes**: For this workshop, we'll run single iterations and use precomputed results to demonstrate the workflow efficiently. In practice, these optimizations would run for many iterations to achieve full convergence.

## Stage 3: Controller Re-tuning with ROSCO

### Why Controller Re-tuning is Essential

Our optimized rotor from Chapter 1 has different aerodynamic characteristics than the original IEA 22MW design. The controller parameters that worked well for the baseline turbine may now be suboptimal or even cause stability issues with our new rotor.

### Controller Optimization Objectives

**Primary Goal**: Minimize tower base damage equivalent loads (DELs) while maintaining operational constraints

**Key Design Variables**:
- **Pitch controller natural frequency (ω)**: Controls how aggressively the pitch system responds
- **Pitch controller damping (ζ)**: Determines system stability and response smoothness  
- **Floating feedback gain (Kp_float)**: Manages platform motion interactions
- **Platform natural frequency (ptfm_freq)**: Used for floating feedback controller filtering

**Constraints**:
- **Rotor overspeed limit**: Ensure safe operation under all conditions
- **Operational stability**: Maintain controller performance across the wind speed range

This optimization uses OpenFAST for high-fidelity aeroelastic analysis, capturing the dynamic interactions between the controller, rotor, tower, and floating platform that are essential for offshore wind turbines.

In [ ]:
# Setting up file paths
stageFolder = "stage-2-controller"

fname_wt_input = os.path.join(WORKSHOP_REPO, "stage-1-aeroStruct","outputs_preCompute", "stage-1-aeroStruct-aero_analysis.yaml")
fname_modeling_options = os.path.join(WORKSHOP_REPO, stageFolder, f"stage-2-controller_modeling.yaml")
fname_analysis_options = os.path.join(WORKSHOP_REPO, stageFolder, f"stage-2-controller_analysis.yaml")


## Controller Optimization Configuration

The controller optimization setup demonstrates WEIS's ability to perform high-fidelity aeroelastic optimization using OpenFAST. This configuration integrates multiple tools for comprehensive dynamic analysis.

### Analysis Options Configuration

The analysis options define our optimization problem:

```yaml
design_variables:
  control:
    servo:
      pitch_control:
          omega:
            flag: True
            min: 0.025
            max: 0.5
          zeta:
            flag: True
            min: 0.1
            max: 3.0
          Kp_float:
            flag: True
            min: -40.0  # -0.25
            max: 0
          ptfm_freq:
            flag: True
            max: 0.5

merit_figure: DEL_TwrBsMyt  # Merit figure of the optimization problem. The options are 'AEP' - 'LCOE' - 'Cp' - 'blade_mass' - 'blade_tip_deflection'

constraints:
  control:
    rotor_overspeed:
      flag: True
      min: 0.0
      max: 0.2

driver:
  optimization:
    flag: True
    tol: 1.e-3            # Optimality tolerance
    max_iter: 200         # Maximum number of iterations (SLSQP)
    maxiter: 60          # Maximum number of iterations (SLSQP)
    solver: LN_COBYLA         # Optimization solver. Other options are 'SLSQP' - 'CONMIN'
    step_size: 1.e-3      # Step size for finite differencing
    form: forward         # Finite differencing mode, either forward or central
```

**Key Configuration Elements:**
- **Merit Figure**: `DEL_TwrBsMyt` - Tower base damage equivalent load in the fore-aft direction
- **Design Variables**: Four controller parameters with physically meaningful bounds
- **Constraints**: Rotor overspeed constraint to ensure safe operation
- **Solver**: LN_COBYLA for gradient-free optimization suitable for noisy objective functions

### Modeling Options Configuration

The modeling options enable high-fidelity simulation through OpenFAST integration:

```yaml
General:
    verbosity: False  # When set to True, the code prints to screen many infos
    openfast_configuration:
        OF_run_fst: stage-2-controller
        save_iterations: True
        save_timeseries: True
        use_exe: True
        allow_fails: True
        fail_value: 9999
        write_stdout: True

WISDEM:
    RotorSE:
        flag: True
        n_pitch_perf_surfaces: 20
        n_tsr_perf_surfaces: 20
        spar_cap_ss: spar_cap_ss
        spar_cap_ps: spar_cap_ps
        peak_thrust_shaving: True
        thrust_shaving_coeff: 0.8
    TowerSE:
        flag: True
        wind: PowerWind  # Wind used
        gamma_f: 1.35    # Safety factor for fatigue loads
        gamma_m: 1.3     # Safety factor for material properties
        gamma_n: 1.0     # Safety factor for ...
        gamma_b: 1.1     # Safety factor for ...
        gamma_fatigue: 1.755  # Safety factor for fatigue loads
        buckling_method: dnvgl # Buckling code type [eurocode or dnvgl]
        buckling_length: 15    # Buckling parameter
        frame3dd:
            shear: True
            geom: True
            tol: 1e-9
    DriveSE:
        flag: True
    FloatingSE:
        flag: True
        symmetric_moorings: True
        gamma_f: 1.35    # Safety factor for fatigue loads
        gamma_m: 1.3     # Safety factor for material properties
        gamma_n: 1.0     # Safety factor for ...
        gamma_b: 1.1     # Safety factor for ...
        gamma_fatigue: 1.755  # Safety factor for fatigue loads
        rank_and_file: True
    BOS:
        flag: True

OpenFAST: # Options for WEIS fidelity level 3 = nonlinear time domain
    flag: True
    simulation:
        DT: 0.01
        CompElast: 1
        CompInflow: 1
        CompAero: 2
        CompServo: 1
        CompHydro: 1
        CompSub: 0
        CompMooring: 3
        CompIce: 0
        OutFileFmt: 3
        NumCrctn: 1
    ElastoDyn:
        FlapDOF1: True
        FlapDOF2: True
        EdgeDOF: True
        TeetDOF: False
        DrTrDOF: False
        GenDOF: True
        YawDOF: False
        TwFADOF1 : True
        TwFADOF2 : True
        TwSSDOF1 : True
        TwSSDOF2 : True
        PtfmSgDOF: True
        PtfmSwDOF: True
        PtfmHvDOF: True
        PtfmRDOF : True
        PtfmPDOF : True
        PtfmYDOF : True
RAFT:
    flag: True
    potential_model_override: 0
    trim_ballast: 2
    heave_tol: 1
    save_designs: True
ROSCO:
    flag: True
    tuning_yaml: ../source/iea22_rosco.yaml

DLC_driver:
    metocean_conditions:
        wind_speed: [4., 6., 8., 10., 12., 14., 16., 18., 20., 22., 24.]
        wave_height_NSS: [0.83, 0.88, 0.94, 1.03, 1.16, 1.34, 1.57, 1.86, 2.22, 2.62, 3.07]
        wave_period_NSS: [6.9, 6.96, 7.02, 7.12, 7.25, 7.43, 7.66, 7.94, 8.27, 8.63, 9.01]
        wave_height_SSS: [6.3, 8, 8, 8.1, 8.5, 8.5, 9.8, 9.8, 9.8, 9.8, 9.9]
        wave_period_SSS: [11.5, 12.7, 12.7, 12.8, 13.1, 13.1, 14.1, 14.1, 14.1, 14.1, 14.1]
        wave_height_1: 6.98
        wave_period_1: 11.7
        wave_height_50: 10.68
        wave_period_50: 14.2
    DLCs:
        - DLC: "1.1"
          n_seeds: 6
          transient_time: 120.0  # 0.
          analysis_time: 600.0  # 10.
```

**Key Integration Elements:**
- **OpenFAST**: Full aeroelastic simulation with all relevant DOFs enabled
- **RAFT**: Frequency-domain analysis for platform initialization
- **ROSCO**: Controller integration with baseline tuning file
- **DLC Driver**: Design Load Case 1.1 (normal operation) with realistic metocean conditions
- **WISDEM**: All structural analysis modules activated for comprehensive modeling

This configuration represents WEIS's highest fidelity level, combining time-domain aeroelastic simulation with controller optimization for floating offshore wind turbines.

In [ ]:
# lets overide a few things so that we dont distrupt the existing results.
analysis_override = {}
analysis_override['general'] = {}
analysis_override['general']['folder_output'] = 'outputsCh1'
analysis_override['driver'] = {}
analysis_override['driver']['optimization'] = {}
analysis_override['driver']['optimization']['max_iter'] = 1 # only run one iteration for this demo, we have precomputed the solutions for you :)


modeling_override = {}
modeling_override['DLC_driver'] = {}
modeling_override['DLC_driver']['DLCs'] = [
    {'DLC': "1.1",
     'n_seeds': 1,
     'wind_speed': [20.0],
     'transient_time': 10.0,
     'analysis_time': 30.0}]

control_20mw, modeling_options, opt_options = weis_main(fname_wt_input, 
                                                 fname_modeling_options, 
                                                 fname_analysis_options,
                                                 analysis_override=analysis_override,
                                                 modeling_override=modeling_override,
                                                 test_run=False
                                                 )

## 2.1: Controller Optimization Results Analysis

Now we'll examine the controller optimization convergence using precomputed results. The optimization minimized tower base damage equivalent loads while satisfying rotor overspeed constraints.

### What to Look For

**Objective Function Behavior:**
- `aeroelastic.DEL_TwrBsMyt`: Tower base fore-aft damage equivalent loads (our primary objective)
- `aeroelastic.rotor_overspeed`: Constraint ensuring safe operational limits

**Controller Parameter Evolution:**
- `tune_rosco_ivc.omega_pc`: Pitch controller natural frequency optimization
- `tune_rosco_ivc.zeta_pc`: Pitch controller damping ratio optimization  
- `tune_rosco_ivc.Kp_float`: Floating platform feedback gain tuning
- `tune_rosco_ivc.ptfm_freq`: Platform frequency parameter for filtering

<!-- **Internal Controller Gains:**
- `sse_tune.tune_rosco.PC_Kp`: Proportional gain from ROSCO tuning
- `sse_tune.tune_rosco.PC_Ki`: Integral gain from ROSCO tuning -->

The convergence plots will show how the optimizer balanced load reduction with operational constraints, demonstrating the effectiveness of controller co-design for floating offshore wind turbines.

In [ ]:
rec_data = load_OMsql(os.path.join(WORKSHOP_REPO, stageFolder,"outputs_preCompute/log_opt.sql")) # not sure what supresses the output here
plot_convergence(
        rec_data, 
        [
                'aeroelastic.DEL_TwrBsMyt', 
                'aeroelastic.rotor_overspeed',  
                'tune_rosco_ivc.Kp_float', 
                'tune_rosco_ivc.omega_pc',
                'tune_rosco_ivc.ptfm_freq', 
                'tune_rosco_ivc.zeta_pc'
        ],
        )

In [ ]:
custom_y_params = [
            ('TwrBsMyt', 'std'),
            ('PtfmPitch', 'max')
]

with open(os.path.join(WORKSHOP_REPO, stageFolder,"outputs_preCompute/openfast_runs/rank_0/iteration_9/summary_stats.p"), 'rb') as f:
    data = pickle.load(f)
plot_weis_summary(data, custom_y_params)

with open(os.path.join(WORKSHOP_REPO, stageFolder,"outputs_preCompute/openfast_runs/rank_0/iteration_69/summary_stats.p"), 'rb') as f:
    data = pickle.load(f)
plot_weis_summary(data, custom_y_params)



## Stage 2: Floating Platform Optimization with RAFT

### Multi-Fidelity Platform Design Approach

Following the methodology from **Zalkind et al. (2024)** "Control Co-Design Studies for a 22 MW Semisubmersible Floating Wind Turbine Platform" (DOI: 10.1088/1742-6596/2767/8/082020), we implement a systematic multi-fidelity approach:

#### Phase 1: RAFT-Based Platform Sizing
- **Tool**: RAFT frequency-domain analysis
- **Purpose**: Rapid initial sizing and optimization of the semisubmersible platform
- **Advantages**: Fast computation allows extensive design space exploration
- **Focus**: Hydrostatic stability, natural periods, and basic motion responses

#### Phase 2: OpenFAST High-Fidelity Refinement  
- **Tool**: OpenFAST time-domain aeroelastic simulation
- **Purpose**: Detailed load analysis and platform optimization with full system interactions
- **Advantages**: Captures nonlinear effects, controller interactions, and realistic loads
- **Focus**: Fatigue loads, extreme responses, and coupled dynamics

#### Phase 3: Integrated Co-Design (Optional)
- **Tool**: OpenFAST time-domain aeroelastic simulation
- **Approach**: Simultaneous optimization of platform and controller parameters
- **Benefits**: Captures synergistic effects between platform design and control strategy
- **Complexity**: Higher computational cost but potentially superior performance

### Platform Design Objectives

**Primary Goals:**
- Minimize platform structural mass (cost reduction)
- Ensure adequate stability margins
- Control platform motions within acceptable limits
- Satisfy manufacturing and installation constraints

**Key Design Variables:**
- Platform geometry (member dimensions, spacing)

This systematic approach demonstrates how WEIS enables efficient platform design by leveraging the appropriate fidelity level for each design phase.

In [ ]:
# Setting up file paths
stageFolder = "stage-3-semisub" # First the RAFT optimization

fname_wt_input = os.path.join(WORKSHOP_REPO, "stage-2-controller","outputs_preCompute", "stage-2-controller.yaml")
fname_modeling_options = os.path.join(WORKSHOP_REPO, stageFolder, f"stage-3-semisub_raft_modeling.yaml")
fname_analysis_options = os.path.join(WORKSHOP_REPO, stageFolder, f"stage-3-semisub_raft_analysis.yaml")

## RAFT Platform Optimization Configuration

### Input File Setup and Analysis Configuration

We're now setting up the RAFT-based floating platform optimization using the controller-optimized turbine from Stage 2. This configuration demonstrates WEIS's ability to perform rapid platform design iterations using frequency-domain analysis.

### Analysis Options Configuration

The analysis configuration defines our semisubmersible platform optimization problem with realistic design variables and constraints:

```yaml
design_variables:
    floating:
        joints:
            flag: True
            z_coordinate:       # draft
                - names: [main_keel, col1_keel, col2_keel, col3_keel]
                  lower_bound: -30.0
                  upper_bound: -12.0
            r_coordinate:
                - names: [col1_keel, col1_freeboard, col2_keel, col2_freeboard, col3_keel, col3_freeboard]
                  lower_bound: 35.0
                  upper_bound: 75.0
        members:
            flag: True
            groups:
                - names: [column1, column2, column3]
                  diameter:
                      lower_bound: 10.0
                      upper_bound: 16.0
                      constant: True

constraints:
    floating:
        survival_heel:
            upper_bound: 0.17453292519943295 # 10 deg
        metacentric_height:
            flag: True
            lower_bound: 1.0 # 15.0 --> Dan's experience
        pitch_period:
            flag: True
            lower_bound: 20. # +/- 10%
            upper_bound: 80.
        heave_period:
            flag: True
            lower_bound: 16. # +/- 10%
            upper_bound: 80.
        fixed_ballast_capacity:
            flag: True
        variable_ballast_capacity:
            flag: True
        freeboard_margin:   # keep freeboard from being submerged below water during survival_heel, largest wave
            flag: True
        draft_margin:   # keep draft from raising above water line during survival_heel, largest wave
            flag: True
        fairlead_depth: # keep fairlead above bottom trough of largest wave
            flag: True
    control:
        Max_PtfmPitch:
            flag: True
            max: 6.0
        nacelle_acceleration:
            flag: True
            max: 0.4 # Change to 2.0 with new raft # prob wont be active with RAFT, maybe change to 0.5 in the future?

merit_figure: structural_mass
```

**Key Design Variables:**
- **Platform Draft (z_coordinate)**: Vertical position of platform elements (-30m to -12m)
- **Column Spacing (r_coordinate)**: Radial distance of columns from center (35m to 75m)  
- **Column Diameter**: Outer diameter of the three main columns (10m to 16m)

<!-- **Critical Constraints:**
- **Survival Heel**: Maximum allowable heel angle during extreme conditions (≤10°)
- **Metacentric Height**: Stability margin ensuring positive restoring moments (≥1.0m)
- **Natural Periods**: Heave (16-80s) and pitch (20-80s) periods to avoid wave resonance
- **Operational Limits**: Platform pitch (≤6°) and nacelle acceleration (≤0.4 m/s²) during operation -->

### Modeling Options Configuration

The modeling configuration enables RAFT's frequency-domain analysis with realistic environmental conditions:

```yaml
RAFT:
    flag: True
    potential_model_override: 0
    trim_ballast: 2
    heave_tol: 1
    save_designs: True
ROSCO:
    flag: True
    tuning_yaml: ../source/iea22_rosco.yaml
DLC_driver:
    metocean_conditions:
        wind_speed: [4., 6., 8., 10., 12., 14., 16., 18., 20., 22., 24.]
        wave_height_NSS: [0.83, 0.88, 0.94, 1.03, 1.16, 1.34, 1.57, 1.86, 2.22, 2.62, 3.07]
        wave_period_NSS: [6.9, 6.96, 7.02, 7.12, 7.25, 7.43, 7.66, 7.94, 8.27, 8.63, 9.01]
        wave_height_SSS: [6.3, 8, 8, 8.1, 8.5, 8.5, 9.8, 9.8, 9.8, 9.8, 9.9]
        wave_period_SSS: [11.5, 12.7, 12.7, 12.8, 13.1, 13.1, 14.1, 14.1, 14.1, 14.1, 14.1]
        wave_height_1: 6.98
        wave_period_1: 11.7
        wave_height_50: 10.68
        wave_period_50: 14.2
    DLCs:
        - DLC: "1.6"
          analysis_time: 600.
          transient_time: 120.
        - DLC: "6.1"
          analysis_time: 600.
          transient_time: 120.
```

**RAFT Configuration:**
- **Potential Model**: 0 setting uses only Morison elements for hydrodynamics
- **Trim Ballast**: Automatically adjusts ballast to maintain zero mean heave
- **Design Load Cases**: DLC 1.6 (normal operation with severe sea state) and DLC 6.1 (parked, with 50-year wind and wave conditions)

<!-- **Environmental Conditions:**
- **Normal Sea States (NSS)**: Operational conditions with moderate waves
- **Severe Sea States (SSS)**: Extreme conditions for survival analysis
- **Wave Heights**: Range from 0.83m to 10.68m covering operational to extreme conditions -->

Since RAFT's frequency-domain analysis has significantly lower computational cost than OpenFAST's time-domain simulation, we can efficiently explore the platform design space while maintaining reasonable execution times for this workshop demonstration.

In [ ]:
# lets overide a few things so that we dont distrupt the existing results.
analysis_override = {}
analysis_override['general'] = {}
analysis_override['general']['folder_output'] = 'outputsCh1'
analysis_override['driver'] = {}
analysis_override['driver']['optimization'] = {}
analysis_override['driver']['optimization']['max_iter'] = 1 # only run one iteration for this demo, we have precomputed the solutions for you :)


control_20mw, modeling_options, opt_options = weis_main(fname_wt_input, 
                                                 fname_modeling_options, 
                                                 fname_analysis_options,
                                                 analysis_override=analysis_override,
                                                #  modeling_override=modeling_override,
                                                 test_run=False
                                                 )

## Results Analysis

### Understanding the RAFT Optimization Convergence

The convergence plots show the evolution of key platform design parameters and constraints during the RAFT-based optimization. This frequency-domain analysis provides rapid insights into platform behavior while maintaining computational efficiency.

### Key Metrics to Analyze

**Platform Motion Responses:**
- **`raft.Max_PtfmPitch`**: Maximum platform pitch angle - should remain within acceptable limits (≤6°)
- **`raft.max_nac_accel`**: Maximum nacelle acceleration - critical for component fatigue (≤0.4 m/s²)
- **`raft.heave_period`**: Platform heave natural period - must avoid wave resonance (16-80s)
- **`raft.pitch_period`**: Platform pitch natural period - must also avoid wave resonance (16-80s)
<!-- 
**Platform Stability and Safety:**
- **`floatingse.metacentric_height_pitch/roll`**: Stability margins ensuring positive restoring moments
- **`floatingse.constr_draft_heel_margin`**: Safety margin against excessive heel during storms
- **`floatingse.constr_freeboard_heel_margin`**: Prevents deck submersion during extreme conditions
- **`floatingse.constr_fairlead_wave`**: Mooring fairlead depth relative to wave action -->

**Platform Structural Design:**
- **`floatingse.system_structural_mass`**: Structural mass of platform (without water ballast) - primary objective to minimize
- **`floating.jointdv_0/1`**: Platform geometry design variables (draft and column spacing)
- **`floating.memgrp1.outer_diameter_in`**: Column diameter optimization

<!-- **Constraint Management:**
- **`floatingse.constr_fixed_margin`**: Fixed ballast capacity constraints
- **`floatingse.constr_variable_margin`**: Variable ballast capacity constraints -->

<!-- ### What to Look For in the Results

**Successful Optimization Indicators:**
1. **Decreasing objective function** (`floatingse.system_structural_mass`) showing mass reduction
2. **Satisfied constraints** with positive margins indicating feasible designs
3. **Stable natural periods** within acceptable ranges avoiding resonance
4. **Controlled motion responses** meeting operational requirements

**Design Trade-offs:**
- Smaller platforms (lower mass) vs. stability requirements
- Column spacing optimization balancing stability and structural efficiency  
- Platform draft affecting both stability and manufacturing constraints -->

This RAFT analysis provides the foundation for subsequent high-fidelity OpenFAST optimization, where time-domain effects and controller interactions will be captured.

In [ ]:
rec_data = load_OMsql(os.path.join(WORKSHOP_REPO, stageFolder,"outputs_preCompute/log_opt.sql")) # not sure what supresses the output here
plot_convergence(
    rec_data, 
    [
        'raft.Max_PtfmPitch', 
        'raft.heave_period', 
        'raft.max_nac_accel', 
        'raft.pitch_period', 
        'floatingse.constr_draft_heel_margin',  # will the bottom of the platform leave the water? 1 = yes
        # 'floatingse.constr_fairlead_wave', 
        # 'floatingse.constr_fixed_margin', 
        # 'floatingse.constr_freeboard_heel_margin', 
        'floatingse.metacentric_height_pitch', 
        # 'floatingse.metacentric_height_roll', 
        'floatingse.constr_variable_margin',   # is there sufficient volume in the variable ballast chambers?  1 = no
        'floatingse.system_structural_mass', 
        'floating.jointdv_0',   # draft
        'floating.jointdv_1',   # column spacing
        'floating.memgrp1.outer_diameter_in'  # outer column diameter
    ]
)





## Stage 3: High-Fidelity Platform Optimization with OpenFAST

### Transitioning from RAFT to OpenFAST Analysis

Having established an initial platform design using RAFT's efficient frequency-domain analysis, we now advance to OpenFAST for high-fidelity time-domain optimization. This represents the second phase of our multi-fidelity platform design methodology.

### Why OpenFAST Refinement is Essential

**Limitations of RAFT Analysis:**
- **Linear Assumptions**: RAFT uses linearized hydrodynamics and simplified aerodynamics
- **No Controller Interactions**: Cannot capture the complex feedback between platform motion and control system
- **Limited Load Assessment**: Frequency-domain analysis misses important nonlinear effects and transient responses
- **Simplified Environmental Modeling**: Less comprehensive representation of wind-wave interactions

**OpenFAST Capabilities:**
- **Nonlinear Time-Domain Simulation**: Captures full system dynamics including large motions and nonlinearities
- **Integrated Controller Analysis**: Includes ROSCO controller interactions with platform motion
- **Comprehensive Load Evaluation**: Full Design Load Case (DLC) analysis for realistic loading conditions
- **Coupled Physics**: Simultaneous aerodynamics, hydrodynamics, structural dynamics, and control system modeling

<!-- ### OpenFAST Optimization Objectives

**Enhanced Design Goals:**
- **Fatigue Load Management**: Minimize damage equivalent loads considering long-term operation
- **Extreme Response Control**: Ensure survival under design storm conditions
- **Controller-Platform Integration**: Optimize platform design considering control system interactions
- **System-Level Performance**: Balance platform cost, motion performance, and overall system efficiency

**Key Differences from RAFT Optimization:**
- **More Detailed Environmental Conditions**: Full DLC suite including turbulent wind and irregular waves
- **Controller Feedback Effects**: Platform motion influences controller behavior, which affects structural loads
- **Nonlinear Dynamics**: Large amplitude motions, mooring nonlinearities, and aerodynamic stall effects
- **Comprehensive Load Channels**: Tower base loads, blade root loads, mooring tensions, and platform accelerations -->

<!-- ### Multi-Fidelity Design Philosophy

This OpenFAST optimization builds upon the RAFT results, using the frequency-domain optimized design as a starting point for high-fidelity refinement. This systematic approach:

1. **Leverages RAFT Efficiency**: Initial design space exploration completed rapidly
2. **Applies OpenFAST Precision**: Detailed optimization with full system physics
3. **Validates Design Assumptions**: Confirms that frequency-domain insights hold under realistic conditions
4. **Refines for Real-World Performance**: Accounts for all relevant physics and operational scenarios -->

This methodology demonstrates WEIS's capability to seamlessly integrate multiple analysis fidelities, enabling both efficient design exploration and detailed engineering validation.

In [ ]:
# Setting up file paths
stageFolder = "stage-3-semisub"

fname_wt_input = os.path.join(WORKSHOP_REPO, stageFolder,"outputs_preCompute", "stage-3-semisub_raft.yaml")
fname_modeling_options = os.path.join(WORKSHOP_REPO, stageFolder, f"stage-3-semisub_of_modeling.yaml")
fname_analysis_options = os.path.join(WORKSHOP_REPO, stageFolder, f"stage-3-semisub_of_analysis.yaml")


## Inputs for OpenFAST Platform Optimization

### Analysis Options Configuration (important parts only):
``` yaml
    control:
        Max_PtfmPitch:
            flag: True
            max: 6.0
        Std_PtfmPitch:
            flag: True
            max: 1.25  # Same as IEA-15MW with same DLCs
        nacelle_acceleration:
            flag: True
            max: 2.0


```

### Modeling Options Configuration (important parts only):
``` yaml
OpenFAST:
    flag: True
[...]
RAFT:
    flag: False
```

In [ ]:
# lets overide a few things so that we dont distrupt the existing results.
analysis_override = {}
analysis_override['general'] = {}
analysis_override['general']['folder_output'] = 'outputsCh1'
analysis_override['driver'] = {}
analysis_override['driver']['optimization'] = {}
analysis_override['driver']['optimization']['max_iter'] = 1 # only run one iteration for this demo, we have precomputed the solutions for you :)

modeling_override = {}
modeling_override['DLC_driver'] = {}
modeling_override['DLC_driver']['DLCs'] = [
    {'DLC': "1.1",
     'n_seeds': 1,
     'wind_speed': [20.0],
     'transient_time': 10.0,
     'analysis_time': 30.0}]

control_20mw, modeling_options, opt_options = weis_main(fname_wt_input, 
                                                 fname_modeling_options, 
                                                 fname_analysis_options,
                                                 analysis_override=analysis_override,
                                                 modeling_override=modeling_override,
                                                 test_run=False
                                                 )


## Analyzing the results

In [ ]:
rec_data = load_OMsql(os.path.join(WORKSHOP_REPO, stageFolder,"outputs_of_preCompute/log_opt.sql")) # not sure what supresses the output here
plot_convergence(
    rec_data, 
    [
        'aeroelastic.Max_PtfmPitch',    # note aeroelastic comes from OpenFAST
        'aeroelastic.max_nac_accel', 
        'floatingse.heave_period', 
        'floatingse.pitch_period', 
        # 'floatingse.constr_draft_heel_margin', 
        'floatingse.constr_fairlead_wave',   # will the fairleads leave the water? 1 = yes
        # 'floatingse.constr_fixed_margin', 
        # 'floatingse.constr_freeboard_heel_margin', 
        # 'floatingse.metacentric_height_pitch', 
        # 'floatingse.metacentric_height_roll', 
        # 'floatingse.constr_variable_margin', 
        'floatingse.system_structural_mass', 
        'floating.jointdv_0', 
        'floating.jointdv_1', 
        'floating.memgrp1.outer_diameter_in'
    ]
)


## Loading in a CCD case

We ran an additional case where we optimized the platform and controller at the same time. This is a more expensive optimization, but it captures the interaction between the platform design and the controller design.

Lets briefly look at the results.

In [ ]:
stageFolder = "stage-3.5-semisubCCD" # First the RAFT optimization

rec_data = load_OMsql(os.path.join(WORKSHOP_REPO, stageFolder,"outputs_of_preCompute/log_opt.sql")) # not sure what supresses the output here
plot_convergence(rec_data, ['aeroelastic.Max_PtfmPitch', 
                            # 'aeroelastic.Std_PtfmPitch', 
                            'aeroelastic.max_nac_accel', 
                            'aeroelastic.rotor_overspeed', 
                            'raft.heave_period', 
                            'raft.pitch_period', 
                            # 'sse_tune.tune_rosco.PC_Ki', 
                            # 'sse_tune.tune_rosco.PC_Kp', 
                            'tune_rosco_ivc.Kp_float', 
                            'tune_rosco_ivc.omega_pc', 
                            'tune_rosco_ivc.ptfm_freq', 
                            'tune_rosco_ivc.zeta_pc', 
                            # 'floatingse.constr_draft_heel_margin', 
                            'floatingse.constr_fairlead_wave', 
                            # 'floatingse.constr_fixed_margin', 
                            # 'floatingse.constr_freeboard_heel_margin', 
                            # 'floatingse.metacentric_height_pitch', 
                            # 'floatingse.metacentric_height_roll', 
                            # 'floatingse.constr_variable_margin', 
                            'floatingse.system_structural_mass', 
                            'floating.jointdv_0', 
                            'floating.jointdv_1', 
                            'floating.memgrp1.outer_diameter_in'])


